In [1]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb") 

wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
%%time
import torch
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# Load dataset
dataset = load_dataset("ag_news")

# Load pre-trained model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)

# Tokenize and preprocess dataset
def preprocess_data(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)  # Adjust max_length as needed

train_dataset = dataset["train"].map(preprocess_data, batched=True)
eval_dataset = dataset["test"].map(preprocess_data, batched=True)

# Freeze lower layers
for param in model.base_model.parameters():
    param.requires_grad = False

    
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    evaluation_strategy="epoch",
    logging_dir="./logs_layer_wise_fine_tuning",
    #report_to= "none"
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train model
trainer.train()

# Evaluate model
trainer.evaluate()

2024-04-29 20:53:47.241659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 20:53:47.241764: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 20:53:47.375835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

Map:   0%|          | 0/7600 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Currently logged in as: anicomanesh. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240429_205422-5zfbbyfo
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run volcanic-forest-15
wandb: ⭐️ View project at https://wandb.ai/anicomanesh/huggingface
wandb: 🚀 View run at https://wandb.ai/anicomanesh/huggingface/runs/5zfbbyfo


Epoch,Training Loss,Validation Loss
1,0.329800,0.295878
2,0.294600,0.287088


CPU times: user 13min 6s, sys: 25.5 s, total: 13min 32s
Wall time: 13min 35s


{'eval_loss': 0.28708797693252563,
 'eval_runtime': 17.6263,
 'eval_samples_per_second': 431.175,
 'eval_steps_per_second': 53.897,
 'epoch': 2.0}